In [1]:
import os

from mindnlp.transformers import AutoTokenizer, AutoModelForCausalLM
from mindnlp.peft import (
    get_peft_model, LNTuningConfig, TaskType)
from mindnlp.dataset import load_dataset
from mindnlp.engine import Trainer, TrainingArguments

[WARNING] ME(9842:124849532983104,MainProcess):2024-05-30-20:13:52.545.26 [mindspore/run_check/_check_version.py:102] MindSpore version 2.2.14 and cuda version 11.2.146 does not match, CUDA version [['10.1', '11.1', '11.6']] are supported by MindSpore officially. Please refer to the installation guide for version matching information: https://www.mindspore.cn/install.
/home/qd/anaconda3/envs/nlpcc/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.243 seconds.
Prefix dict has been built successfully.


In [3]:
classes = ["Unlabeled", "complaint", "no complaint"]
model_name_or_path = "bigscience/bloomz-560m"

dataset_name = "twitter_complaints"
max_length = 64
lr = 1e-4
num_epochs = 25
batch_size = 8

In [4]:
peft_config = LNTuningConfig(task_type=TaskType.CAUSAL_LM)

In [5]:
dataset = load_dataset("ought/raft", dataset_name, trust_remote_code=True)

/home/qd/anaconda3/envs/nlpcc/lib/python3.9/site-packages/datasets/load.py:1486: FutureWarning: The repository for ought/raft contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ought/raft
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [6]:
dataset['train'] = dataset['train'].project(['Tweet text', 'Label'])
dataset['test'] = dataset['test'].project(['Tweet text', 'Label'])

In [7]:
dataset["train"].get_col_names()

['Tweet text', 'Label']

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
tokenizer.eos_token_id, tokenizer.pad_token_id

(2, 3)

In [9]:
target_max_length = max(tokenizer(classes, return_length=True)["length"])
target_max_length

3

In [10]:
def num_label_2_text_label(num_label):
    return classes[num_label]


def concate_text_label(texts, labels):
    labels = f"{str(labels)}</s>"
    text_formatted = f"Tweet text : {texts} Label : {labels}"
    output = tokenizer(
        text_formatted,
        text_target=labels,
        max_length=max_length,
        padding="max_length",
        truncation=True
    )

    output["labels"] = [-100 if x == tokenizer.pad_token_id else x for x in output['labels']]

    return output["input_ids"], output["attention_mask"], output['labels']


def datapipe(data):
    data = data.map(
        num_label_2_text_label,
        input_columns="Label",
        output_columns="labels",
    )
    data = data.map(
        concate_text_label,
        input_columns=["Tweet text", "labels"],
        output_columns=["input_ids", "attention_mask", "labels"],
    )
    return data

In [11]:
dataset["train"] = datapipe(dataset["train"])
dataset['test'] = datapipe(dataset["test"])

In [12]:
dataset["train"].get_col_names()

['input_ids', 'attention_mask', 'labels']

In [13]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 100,352 || all params: 559,314,944 || trainable%: 0.017941948642087417


In [14]:
training_args = TrainingArguments(
    output_dir="./output",
    num_train_epochs=num_epochs,
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="epoch",
    save_strategy="epoch",
)

In [15]:
trainer = Trainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    args=training_args,
)

In [16]:
trainer.train()

                                               
  4%|▍         | 7/175 [00:17<00:50,  3.34it/s]  

{'eval_loss': 4.95366907119751, 'eval_runtime': 15.0169, 'eval_samples_per_second': 28.302, 'eval_steps_per_second': 3.596, 'epoch': 1.0}


                                                
  8%|▊         | 14/175 [00:36<02:19,  1.15it/s] 

{'eval_loss': 4.8654632568359375, 'eval_runtime': 14.9481, 'eval_samples_per_second': 28.432, 'eval_steps_per_second': 3.613, 'epoch': 2.0}


                                                
 12%|█▏        | 21/175 [00:54<02:20,  1.10it/s] 

...

                                                 
100%|██████████| 175/175 [07:50<00:00,  1.08it/s]

{'eval_loss': 4.167085647583008, 'eval_runtime': 15.1478, 'eval_samples_per_second': 28.057, 'eval_steps_per_second': 3.565, 'epoch': 25.0}


100%|██████████| 175/175 [07:52<00:00,  2.70s/it]

{'train_runtime': 472.5518, 'train_samples_per_second': 2.963, 'train_steps_per_second': 0.37, 'train_loss': 6.5912158203125, 'epoch': 25.0}


TrainOutput(global_step=175, training_loss=6.5912158203125, metrics={'train_runtime': 472.5518, 'train_samples_per_second': 2.963, 'train_steps_per_second': 0.37, 'train_loss': 6.5912158203125, 'epoch': 25.0})

In [17]:
# 'twitter_complaints_bigscience_bloomz-560m_LN_TUNING_CAUSAL_LM'
peft_model_id = f"{dataset_name}_{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}".replace(
    "/", "_"
)
model.save_pretrained(peft_model_id)

In [18]:
peft_model_id

'twitter_complaints_bigscience_bloomz-560m_LN_TUNING_CAUSAL_LM'

In [19]:
from mindnlp.peft import PeftModel, PeftConfig

# load the LNTuning config
config = PeftConfig.from_pretrained(peft_model_id)
# load the base LM
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
# merge LNTuning weights into the base LM
model = PeftModel.from_pretrained(model, peft_model_id)

In [20]:
text = "Tweet text : I am very angry with the service provided by the company Label :"

In [23]:
output = tokenizer(text, return_tensors='ms')
output

{'input_ids': Tensor(shape=[1, 16], dtype=Int64, value=
[[227985,   5484,    915 ...  16333,  77658,    915]]), 'attention_mask': Tensor(shape=[1, 16], dtype=Int64, value=
[[1, 1, 1 ... 1, 1, 1]])}

In [31]:
out = model.generate(
        input_ids=output["input_ids"], attention_mask=output["attention_mask"], max_new_tokens=3, eos_token_id=3
    )

In [32]:
tokenizer.decode(out[0])

'Tweet text : I am very angry with the service provided by the company Label : it is not'